# Yara Playground

This notebook allows us to run a selection of Yara rules against a collection of files. This comes in handy when we use it in combination with previously downloaded files using notebooks like `virustotal_vba_analysis` which will download and store samples, and pass them in a stored variable `stored` for use in this notebook.

The input format of the `stored` variable would be:

```
stored = [{'path':'/tmp/samples/SHAHASH','name':'name of sample','hash':'SHA1HASH'},...}
```

In [ ]:
%store -r stored

RESULTS_FOUND=False

try:
    if len(stored) > 0:
        print("A list of stored results has been found for use in this notebook!!")
        for result in stored:
            print(f"Path: {result['path']}")
                  
        RESULTS_FOUND=True
    else:
        print("No stored results have been found... make sure you provide sample input below")
except NameError:
    print("No stored results have been found... make sure you provide sample input below")

Optional step of importing a list of rules for later use (only needed if we aren't crafting rules on the fly).

In [ ]:
!git clone --depth=1 https://github.com/Yara-Rules/rules.git /tmp/rules

If we are crafting our own rules, we can use run the step below which will give us an input field to work with.

In [ ]:
import sys
sys.path.append('../src/')
import ipywidgets as widgets

yaraInput = widgets.Textarea(
    value='rule dummy { \n condition: true\n}',
    placeholder='Yara Rule Goes Here...',
    description='Yara Rule:',
    disabled=False,
    rows=10,
    columns=40
)
display(yaraInput)

Now we can apply our rules to previously downloaded VT samples, which should speed up the process of tuning Yara rules to the latest downloaded samples.

In [ ]:
from helpers import yarahelper
import ipywidgets as widgets
import pandas as pd

#m = yarahelper.YaraMatch('/tmp/rules/malware/GEN_PowerShell.yar')
m = yarahelper.YaraMatch.usingYaraString(yaraInput.value)

results = []

if RESULTS_FOUND:
    for result in stored:
        matches = m.getAllMatches(result["path"])
        if len(matches) != 0:
            results.append((matches, result["name"], result["hash"]))
            # Display our results
    pd.set_option('max_colwidth', 800)
    df = pd.DataFrame(data=results, columns=["Yara Name","Name","Hash"])
    display(df)
else:
    paths = ["/tmp/samples/test"]
    for result in paths:
        matches = m.getAllMatches(result)
        if len(matches) != 0:
            results.append((matches, result))
            # Display our results
    pd.set_option('max_colwidth', 800)
    df = pd.DataFrame(data=results, columns=["Yara Name","Path"])
    display(df)
        
